In [3]:
#!/usr/bin/env python
# coding: utf-8

import pandas as pd
import statsmodels.api as sm
import numpy as np
from tensorflow import keras
import os
from sklearn.preprocessing import StandardScaler
os.environ['KMP_DUPLICATE_LIB_OK']='True'

NN3 = keras.saving.load_model('model\\NN3')

from flask import Flask, render_template, request
app = Flask(__name__)

def check_params(x1, x2):
    message = f"X1 = {x1}, X2 = {x2}"
    return message

def print_params_for_NN(x1, x2, x3):
    if (x1 == "" or x2 == "" or x3 == ""):
        message = "ОШИБКА! Вы не ввели параметры."
    elif(float(x1) >= 0 and float(x2) >=0 and float(x3) >=0):
        message = f"Работники, чел.: {x1}\nНаличие тракторов, шт.: {x2}\nОбщая площадь земли, га: {x3}"
    else:
        message = "ОШИБКА! Введенные значения должны быть больше или равны 0."
    return message

def calculate_NN(x1, x2, x3):
    X = pd.DataFrame([[x1,x2,x3]],columns=['Работники, чел.','Наличие тракторов, шт.','Общая площадь земли, га'])
    standardscaler = StandardScaler()
    transform_X = standardscaler.fit_transform(X)
    res = NN3.predict(transform_X)
    # create empty table with 3 fields
    Predict_dataset_like = np.zeros(shape=(len(X), 3) )
    # put the predicted values in the right field
    Predict_dataset_like[:,0] = res[:,0]
    # inverse transform and then select the right field
    res_inverse = standardscaler.inverse_transform(Predict_dataset_like)[:,0]
    message = ": ".join(["Доход, тыс. руб.", str(res_inverse[0])])
    return message

@app.route("/", methods=["post", "get"])
def index():
    message = ''
    message2 = ''
    x1=0
    x2=0
    x3=0
    if request.method == "POST":
        x1 = request.form.get("x1")
        x2 = request.form.get("x2")
        x3 = request.form.get("x3")
        
        message = print_params_for_NN(x1, x2, x3) 
        if "ОШИБКА" not in message:
            message2 = calculate_NN(float(x1), float(x2), float(x3))
        
    return render_template("index.html", message=message, message2=message2, x1=x1, x2=x2, x3=x3)

app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [25/Apr/2023 18:03:28] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2023 18:03:30] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2023 18:03:39] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2023 18:03:51] "POST / HTTP/1.1" 200 -


1/1 [==============================] - 0s 62ms/step


127.0.0.1 - - [25/Apr/2023 18:04:01] "POST / HTTP/1.1" 200 -
